In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline

In [ ]:
# NV levels transitions

nvlevels.get_transitions_ExEy(80.3,90.8,
                              show_ms0_transitions=True,
                              show_m1_transitions=True,
                              show_p1_transitions=False,
                              show_A_transitions=False,
                              show_FB_A_transitions=False,
                              show_FB_E_transitions=False,
                              show_E_prime_flip_transitions=False)

In [ ]:
### Fitting laserscans 1 peak at a time

f_min = 85
f_max = 100
g_a= 100
g_A= 300
g_x0 = 91
g_gamma = 0.5
fixed=[]


folder = r'D:\measuring\data\20161005\134741_laser_scan_Harry_Scan5_NV4_g_0.0_r_2.0'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
if len(d)>2:
    voltage, frequency, counts = d[:,0],d[:,1],d[:,2]

fig,ax=plt.subplots(1, figsize=(16,5))
plt.plot(frequency,counts,'-')
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
ax.set_ylabel("Countrate (Hz)", fontsize = 14)
plt.savefig(os.path.join(folder,'Scan5_NV2_wholefigure.png'))

idx_min = np.searchsorted(frequency, f_min, side="left")
idx_max = np.searchsorted(frequency, f_max, side="left")
frequency = frequency[idx_min:idx_max]
counts = counts[idx_min:idx_max]

fig,ax=plt.subplots(1, figsize=(8,5))
    
p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
fit_result = fit.fit1d(frequency,counts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

T = fit_result['params_dict']['x0']
gamma = fit_result['params_dict']['gamma']
u_gamma = fit_result['error_dict']['gamma']

print 'T = ', T , 'GHz'
print 'Linewidth T', gamma, 'GHz +/-', u_gamma, 'GHz'


plot.plot_fit1d(fit_result, np.linspace(frequency[0],frequency[-1],len(frequency)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='red', data_linestyle = '-', print_info= False)
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
ax.set_ylabel("Countrate (Hz)", fontsize = 14)
ax.legend(title='Transition at %.2f GHz, Linewidth = %.2f $\pm$ %.2f MHz'%(T, gamma*1.e3,u_gamma*1.e3))
plt.savefig(os.path.join(folder,'Scan5_NV4_Ex.png'))


In [ ]:
# plotting optical depth versus linewidth Ey

folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'
fig,ax=plt.subplots(1, figsize=(8,5))
od = [0.6, 1, 2, 1.8]
Ey = [315, 367, 421, 295]
yerr = [13, 12, 102, 12]

eb = plt.errorbar(od, Ey, yerr=yerr, fmt='.', color='b')
ax.set_xlabel("Optical depth (um)", fontsize = 14)
ax.set_ylabel("Linewidth Ey (MHz)", fontsize = 14)
ax.set_xlim(0,2.5)
plt.savefig(os.path.join(folder,'OpticalDepth_vs_Eylinewidth.png'))


strain = [6.99,7.1,18.36,7.36,7.84,18.8,20.49,13.49,14.52]
avg_strain = np.mean(strain)
print avg_strain

In [ ]:
### Plot 2 graphs in one figure with two axes 

from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import matplotlib.pyplot as plt

folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'
od = [0.6, 1, 2, 1.8]
Ey = [315, 367, 421, 295]
Ex = [522, 619, 1417, 239]
yerr = [13, 12, 102, 12]
xerr = [38,57,384,65]

if 1:

    host = host_subplot(111, axes_class=AA.Axes)
    plt.subplots_adjust(right=0.75)

    par1 = host.twinx()
    
    offset = 60

    host.set_xlim(0, 2.5)
    host.set_ylim(0, 1500)

    host.set_xlabel("Optical depth (um)")
    host.set_ylabel("Intrinsic linewidth Ey (Mhz)")
    par1.set_ylabel("Linewidth Ex with green excitation (Mhz)")
    

    p1, = host.plot(od, Ey, '.')
    p2, = par1.plot(od, Ex, '.')
    eb1 = plt.errorbar(od, Ey, yerr=yerr, fmt='.', color='b')
    eb2 = plt.errorbar(od, Ex, yerr=xerr, fmt='.', color='g')
    
    par1.set_ylim(0, 1500)

    host.legend()

    host.axis["left"].label.set_color(p1.get_color())
    par1.axis["right"].label.set_color(p2.get_color())

    plt.draw()
    #plt.show()
    plt.savefig(os.path.join(folder,'OpticalDepth_vs_Eylinewidth_Ex_green.png'))

In [ ]:
### Plotting two graphs in one figure same axes

import matplotlib.pyplot as plt
%matplotlib inline

import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep
data_dir = 'D:\measuring\data/20161004/Sophie_ZPL_spectrometer/20160920 ZPL Sophie Area 5'
data_dir2 = 'D:\measuring\data/20161004/Harry_ZPL_spectrometer'

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data(min_frq=470,max_frq=473)


intensity = intensities[:,0]
intensity = intensity - 600

sa_2D_2 = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir2,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies2,filenumbers2,intensities2 = sa_2D_2.get_data(min_frq=470,max_frq=473)

intensity2 = intensities2[:,0]

fig,ax=plt.subplots(1, figsize=(10,5))
plt.plot(frequencies,intensity, '-', label='d = 3 um')
plt.plot(frequencies2, intensity2, '-', label='d = 27 um')

plt.xlabel('Frequency (THz)')
plt.ylabel('Intensity (a.u.)')
ax.set_ylim(0,5000)
#ax.set_yscale('log')
plt.legend()
plt.title('Sophie"s ZPL is translated down 600, for clarity')
plt.draw()
#plt.show()
folder = 'D:\measuring\data/20161004'
plt.savefig(os.path.join(folder,'ZPL_Sophie_Harry_Spectrometer.png'))



In [ ]:
### Plotting and fitting spectrometer data

import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep
data_dir = 'D:\measuring\data/20161004/Sophie_ZPL_spectrometer/20160920 ZPL Sophie Area 5'

f_min = 0
f_max = 1000
g_a= 0
g_A= 100
g_x0 = 471.7
g_gamma = 0.5
fixed=[]

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data(min_frq=470,max_frq=473)
reload(spectrometer_analysis_2Dsweep)
intensity = intensities[:,0]


fig,ax=plt.subplots(1, figsize=(16,5))
plt.plot(frequencies,intensities[:,0],'-')
ax.set_xlabel("Frequency (THz)", fontsize = 14)
ax.set_ylabel("Intensity(a.u.)", fontsize = 14)
plt.savefig(os.path.join(data_dir,'Sophie_Scan5_ZPL.png'))

idx_min = np.searchsorted(frequencies, f_min, side="left")
idx_max = np.searchsorted(frequencies, f_max, side="left")
frequency = frequencies[idx_min:idx_max]
intensity = intensity[idx_min:idx_max]
print max(frequency)
print len(frequency)


fig,ax=plt.subplots(1, figsize=(8,5))
    
p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
fit_result = fit.fit1d(frequency,intensity, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

T = fit_result['params_dict']['x0']
gamma = fit_result['params_dict']['gamma']
u_gamma = fit_result['error_dict']['gamma']

print 'T = ', T , 'GHz'
print 'Linewidth T', gamma, 'GHz +/-', u_gamma, 'GHz'


plot.plot_fit1d(fit_result, np.linspace(frequency[0],frequency[-1],len(frequency)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='red', data_linestyle = '-', print_info= False)
ax.set_xlabel("Frequency (THz)", fontsize = 14)
ax.set_ylabel("Intensity (a.u.)", fontsize = 14)
ax.set_yscale('log')
ax.legend(title='ZPL at %.2f THz, Linewidth = %.2f $\pm$ %.2f GHz'%(T, gamma*1.e3,u_gamma*1.e3))
plt.savefig(os.path.join(data_dir,'fit_Sophie_scan5.png'))

In [ ]:
# plotting microwave power versus linewidth Ex

folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans'
fig,ax=plt.subplots(1, figsize=(8,5))
mw = [10, 10,12,15,15,20,26]
Ex = [975, 953,975,912,993,1284,2774]
yerr = [53,50,67,63,64,91,108]

eb = plt.errorbar(mw, Ex, yerr=yerr, fmt='.', color='b')
ax.set_xlabel("Microwave power (dBm)", fontsize = 14)
ax.set_ylabel("Linewidth Ex (MHz)", fontsize = 14)
ax.set_xlim(5,30)
plt.savefig(os.path.join(folder,'Microwavepower_vs_Exlinewidth_Scan5NV4.png'))

